# siamese NN

## build layer 1

In [1]:
# !pip install opencv-python
# !pip install matplotlib
# !pip install numpy

In [2]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# !pip install tensorflow

In [4]:
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

### build the first part of snn

In [5]:
def first_part_layer():
    inp = Input(shape=(105,105,3), name='input image')
    
    # block 1
    c1 = Conv2D(64, (10,10), activation='relu', name='c1')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same', name='m1')(c1)
                      
    # block 2            
    c2 = Conv2D(128, (7,7), activation='relu', name='c2')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same', name='m2')(c2)
                      
    # block 3
    c3 = Conv2D(128, (4,4), activation='relu', name='c3')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same', name='m3')(c3)
                      
    # block 4
    c4 = Conv2D(256, (4,4), activation='relu', name='c4')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid', name='d1')(f1)
                      
    return Model(inputs=[inp],outputs=[d1] ,name='make_first_layer')

In [6]:
first_part = first_part_layer()
first_part.summary()

Model: "make_first_layer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 c1 (Conv2D)                 (None, 96, 96, 64)        19264     
                                                                 
 m1 (MaxPooling2D)           (None, 48, 48, 64)        0         
                                                                 
 c2 (Conv2D)                 (None, 42, 42, 128)       401536    
                                                                 
 m2 (MaxPooling2D)           (None, 21, 21, 128)       0         
                                                                 
 c3 (Conv2D)                 (None, 18, 18, 128)       262272    
                                                                 
 m3 (MaxPooling2D)           (None, 9, 9, 128)    

#### reate the distance class to comapre the two inputs

In [7]:
# didtatnce class
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
    def call(self, input_mode, validation_mode):
        return tf.math.abs(input_mode - validation_mode)

## build the model

In [8]:
def seamise_model():
    #
    input_image = Input(shape=(105,105,3), name='input image')
    
    #
    validation_image = Input(shape=(105,105,3), name='validation image')
    
    # combine
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(first_part(input_image), first_part(validation_image))
    
    # add dence layer 
    
    classifier = Dense(1, activation = 'sigmoid')(distances)
    
    return  Model(inputs=[input_image, validation_image] ,outputs = classifier, name='SiameseNetwork')
    

In [9]:
Siamese_Network = seamise_model()

In [10]:
Siamese_Network.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input image (InputLayer)       [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation image (InputLayer)  [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 make_first_layer (Functional)  (None, 4096)         38960448    ['input image[0][0]',            
                                                                  'validation image[0